In [30]:
import pandas as pd
import sqlite3
import holoviews as hv
import matplotlib as pyplot
import plotly.graph_objects as go
import plotly.express as pex
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'plotly'

In [2]:
hop_proj=sqlite3.connect('hop_proj.sqlite')

for chunk in tqdm(pd.read_csv('nppes_with_taxonomy.csv', dtype = {'postal_code':str}, chunksize = 1000)):
    chunk['postal_code'] = chunk['postal_code'].str.slice(stop=5)
    chunk.to_sql('nppes', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('nucc_taxonomy_220.csv', chunksize = 1000)):
    chunk.to_sql('taxonomy', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('ZIP_CBSA_122021.csv', chunksize = 1000, )):
    chunk['zip'] = chunk['zip'].str.slice(start = 1, stop = 6)
    chunk.to_sql('cbsa', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('hoptrim.csv', chunksize = 1000)):
    chunk.to_sql('hop', hop_proj, if_exists = 'append', index = False)

hop_proj.execute('CREATE INDEX npi ON nppes(npi)')

hop_proj.execute('CREATE INDEX postal_code ON nppes(postal_code)')

hop_proj.execute('CREATE INDEX primary_taxonomy ON nppes(primary_taxonomy)')

hop_proj.execute('CREATE INDEX from_npi ON hop(from_npi)')

hop_proj.execute('CREATE INDEX to_npi ON hop(to_npi)')

hop_proj.execute('CREATE INDEX zip ON cbsa(zip)')

hop_proj.execute('CREATE INDEX code ON taxonomy(code)')

hop_proj.close()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [4]:
hop_proj=sqlite3.connect('hop_proj.sqlite')

query = '''SELECT h.from_npi,
            h.to_npi,
            h.transaction_count,
            h.average_day_wait,
            h.std_day_wait,
            t.code,
            t.grouping,
            t.classification,
            t.specialization,
            t.class_spec,
            n.npi,
            n.entity_code,
            n.org_name,
            n.last_name,
            n.first_name,
            n.city,
            n.postal_code,
            n.primary_taxonomy
             FROM hop as h
             INNER JOIN nppes as n
             ON h.from_npi = n.npi
             INNER JOIN taxonomy as t
             ON t.code = n.primary_taxonomy
             WHERE state = 'TN'
             '''

hop_test = pd.read_sql(query, hop_proj)

hop_proj.close()

In [5]:
hop_test

,from_npi,to_npi,transaction_count,average_day_wait,std_day_wait,code,grouping,classification,specialization,class_spec,npi,entity_code,org_name,last_name,first_name,city,postal_code,primary_taxonomy
0,1841293891,1245290857,231,28.857,60.036,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,Surgery,1841293891,1.0,None,GIBBS,ELMER,CROSSVILLE,38555,208600000X
1,1841293891,1245437060,50,35.260,60.604,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,Surgery,1841293891,1.0,None,GIBBS,ELMER,CROSSVILLE,38555,208600000X
2,1841293891,1265401483,72,10.347,34.328,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,Surgery,1841293891,1.0,None,GIBBS,ELMER,CROSSVILLE,38555,208600000X
3,1841293891,1457302010,366,3.101,19.513,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,Surgery,1841293891,1.0,None,GIBBS,ELMER,CROSSVILLE,38555,208600000X
4,1841293891,1467598896,107,41.271,70.229,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,Surgery,1841293891,1.0,None,GIBBS,ELMER,CROSSVILLE,38555,208600000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831613,1053881706,1164488219,86,0.000,0.000,367500000X,Physician Assistants & Advanced Practice Nursi...,"Nurse Anesthetist, Certified Registered",None,"Nurse Anesthetist, Certified Registered",1053881706,2.0,"TENNESSEE VALLEY ANESTHESIA ASSOCIATES, LLC",None,None,JOHNSON CITY,37604,367500000X
831614,1053881706,1346271251,52,0.000,0.000,367500000X,Physician Assistants & Advanced Practice Nursi...,"Nurse Anesthetist, Certified Registered",None,"Nurse Anesthetist, Certified Registered",1053881706,2.0,"TENNESSEE VALLEY ANESTHESIA ASSOCIATES, LLC",None,None,JOHNSON CITY,37604,367500000X
831615,1053881706,1578529632,85,0.000,0.000,367500000X,Physician Assistants & Advanced Practice Nursi...,"Nurse Anesthetist, Certified Registered",None,"Nurse Anesthetist, Certified Registered",1053881706,2.0,"TENNESSEE VALLEY ANESTHESIA ASSOCIATES, LLC",None,None,JOHNSON CITY,37604,367500000X
831616,1053881706,1659574127,51,0.000,0.000,367500000X,Physician Assistants & Advanced Practice Nursi...,"Nurse Anesthetist, Certified Registered",None,"Nurse Anesthetist, Certified Registered",1053881706,2.0,"TENNESSEE VALLEY ANESTHESIA ASSOCIATES, LLC",None,None,JOHNSON CITY,37604,367500000X


In [4]:
hop_proj=sqlite3.connect('hop_proj.sqlite')

query = '''SELECT *
           FROM nppes
           WHERE state = 'TN'
           LIMIT 10'''
                  

hop_query = pd.read_sql(query, hop_proj)

hop_proj.close()

,npi,entity_code,org_name,last_name,first_name,middle,prefix,suffix,credential,address1,address2,city,state,postal_code,primary_taxonomy
0,1841293891,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,49 CLEVELAND ST 310,None,CROSSVILLE,TN,38555,208600000X
1,1659374601,1.0,None,OBERDICK,WENDY,TIPTON,None,None,MD,105 W STONE DR,STE 1F,KINGSPORT,TN,37660,207Q00000X
2,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,37203,363L00000X
3,1003819046,1.0,None,NYLANDER,BARBARA,H,None,None,M.D.,345 23RD AVE N,SUITE 209,NASHVILLE,TN,37203,207VG0400X
4,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,37055,207Q00000X
5,1396748323,1.0,None,CAMPBELL,LAURA,IVEY,DR.,None,PHARMD,272 HIGHWAY 11 E,None,BULLS GAP,TN,37711,183500000X
6,1922001957,1.0,None,PRESLEY,RICHARD,E,None,None,M.D.,2011 MURPHY AVE,STE 302,NASHVILLE,TN,37203,207V00000X
7,1104829134,1.0,None,FALCONER,RANDALL,None,DR.,None,MD,1728 N EASTMAN RD,None,KINGSPORT,TN,37664,207Y00000X
8,1073516001,1.0,None,ROSS,DAVID,L,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,37055,207R00000X
9,1700889730,1.0,None,WEISS,KENNETH,S,None,None,M.D.,6286 BRIARCREST AVE STE 200,None,MEMPHIS,TN,38120,207XX0005X


In [ ]:
hop_proj=sqlite3.connect('hop_proj.sqlite')

tst_query = '''SELECT *
                FROM hop as h
                INNER JOIN nppes as n
                ON h.from_npi = n.npi
                '''

test_query = pd.read_sql(tst_query, hop_proj)

hop_proj.close()

In [ ]:
test_query

In [2]:
bigreferrals = pd.read_csv("referrals_big.csv")

In [3]:
bigreferrals.columns

Index(['from_npi', 'referral_id', 'entity_from', 'name_from',
       'organization_from', 'taxonomy_code_from', 'classification_from',
       'specialization_from', 'address_line1_from', 'address_line2_from',
       'city_from', 'state_from', 'zip_from', 'to_npi', 'entity_to', 'name_to',
       'organization_to', 'taxonomy_code_to', 'classification_to',
       'specialization_to', 'address_line1_to', 'address_line2_to', 'city_to',
       'state_to', 'zip_to', 'patient_count', 'transaction_count',
       'average_day_wait', 'std_day_wait', 'address_from', 'address_to'],
      dtype='object')

In [9]:
specialization_referrals_grouped 

NameError: name 'specialization_referrals_grouped' is not defined

In [11]:
specialization = bigreferrals[['specialization_from', 'organization_to', 'patient_count']]

In [15]:
specialization_referrals_grouped = specialization.groupby(by = ['specialization_from',
                                                                'organization_to']).sum()[['patient_count']].reset_index()

In [28]:
specialization_referrals_grouped = specialization_referrals_grouped.iloc[0:20]
specialization_referrals_grouped

,specialization_from,organization_to,patient_count
0,Acute Care,ACLA PC,562
1,Acute Care,"ADVANCED DIAGNOSTIC IMAGING, PC",5211
2,Acute Care,AEGIS SCIENCES CORPORATION,782
3,Acute Care,"ALIVE HOSPICE, INC.",55
4,Acute Care,"AMBULANCE SERVICE OF MURFREESBORO, LLC",46
5,Acute Care,"ANESTHESIA MEDICAL GROUP, PC",1160
6,Acute Care,ANESTHESIA SERVICES ASSOCIATES PLLC,1052
7,Acute Care,"APP OF TENNESSEE HM, PLLC",184
8,Acute Care,"ASPIRE HEALTH MEDICAL PARTNERS, PC",116
9,Acute Care,ASSOCIATED PATHOLOGISTS LLC,569


In [27]:
hv.Sankey(specialization_referrals_grouped, 
          kdims =['specialization_from', 'organization_to'],
          vdims = ['patient_count'])

:Sankey   [specialization_from,organization_to]   (patient_count)

In [26]:
from_vandy = bigreferrals[bigreferrals['organization_from'] == 'VANDERBILT UNIVERSITY MEDICAL CENTER']

In [ ]:
bigreferrals[bigreferrals['classification_from'] == 'Psychiatric Unit']

In [43]:
from_vandy[from_vandy['classification_from'] == 'Pathology']

,from_npi,referral_id,entity_from,name_from,organization_from,taxonomy_code_from,classification_from,specialization_from,address_line1_from,address_line2_from,...,address_line2_to,city_to,state_to,zip_to,patient_count,transaction_count,average_day_wait,std_day_wait,address_from,address_to


In [45]:
from_vandy.classification_from.unique()

array(['Internal Medicine', 'General Acute Care Hospital', 'Pharmacy',
       'Psychiatric Unit', 'Rehabilitation Unit', 'Clinic/Center'],
      dtype=object)